# CS486 - Artificial Intelligence
## Lesson 24 - Dynamic Bayes Networks

Hidden Markov Models operate on the assumption that there are no dependencies 

In [6]:
import helpers
from aima.probability import *
from aima.notebook import psource